<h1>Q1

In [2]:
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import requests

from geopy.geocoders import Nominatim
import folium

from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
res = requests.get(url)
print(res.status_code)

200


Using bs4 to retrive data

In [4]:
soup = BeautifulSoup(res.content, 'html.parser')

tab = soup.find('table')
tab_row = []

tab_tr = tab.find_all('tr')

for tr in tab_tr:
    aux = tr.find_all('td')
    if aux:
        tab_row.append(aux)
        
tab_row[:2]

[[<td>M1A
  </td>,
  <td>Not assigned
  </td>,
  <td>Not assigned
  </td>],
 [<td>M2A
  </td>,
  <td>Not assigned
  </td>,
  <td>Not assigned
  </td>]]

Cleaning Data

In [5]:
data = []

for row in tab_row:
    poco = row[0].text.rstrip()
    boro = row[1].text.rstrip()
    nbhd = row[2].text.rstrip()
    
    if boro != 'Not assigned':
        if nbhd == 'Not assigned':
            nbhd = borough
        data.append([poco, boro, nbhd])

data[:2]

[['M3A', 'North York', 'Parkwoods'], ['M4A', 'North York', 'Victoria Village']]

<h1>Answer to Q1

In [6]:
columns = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(data, columns=columns)
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


<h1>Q2

In [7]:
df_geo=pd.read_csv(r'D:/Datasets/Geospatial_Coordinates.csv')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
dicto={'Postal Code': 'PostalCode'}
df_geo.rename(columns=dicto, inplace=True)
df_geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h1>Answer to Q2

In [9]:
df_concat = pd.merge(df, df_geo, on="PostalCode", how='left')
df_concat

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [10]:
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PostalCode    103 non-null    object 
 1   Borough       103 non-null    object 
 2   Neighborhood  103 non-null    object 
 3   Latitude      103 non-null    float64
 4   Longitude     103 non-null    float64
dtypes: float64(2), object(3)
memory usage: 4.8+ KB


<h1>Q3

Using Foursquare API

In [11]:
CLIENT_ID = '...' # your Foursquare ID
CLIENT_SECRET = '...' # your Foursquare Secret
ACCESS_TOKEN = '...' # your FourSquare Access Token
#Code=...
VERSION = '20180604'

In [12]:
neighborhood_latitude = df_concat.loc[4,'Latitude']
neighborhood_longitude = df_concat.loc[4,'Longitude']

In [13]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [14]:
results = requests.get(url).json()

In [15]:
venues = results['response']['groups'][0]['items']

In [19]:
nearby_venues = pd.json_normalize(venues) 
nearby_venues

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,...,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,e-0-4b9d206bf964a520e69136e3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4b9d206bf964a520e69136e3,Queen's Park,University Ave.,at Wellesley Ave.,43.663946,-79.392180,"[{'label': 'display', 'lat': 43.66394609897775...",...,M5R 2E8,CA,Toronto,ON,Canada,"[University Ave. (at Wellesley Ave.), Toronto ...","[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",0,[],NaN
1,e-0-4a8355bff964a520d3fa1fe3-1,0,"[{'summary': 'This spot is popular', 'type': '...",4a8355bff964a520d3fa1fe3,Mercatto,101 College St,NaN,43.660391,-79.387664,"[{'label': 'display', 'lat': 43.66039091189854...",...,M5G,CA,Toronto,ON,Canada,"[101 College St, Toronto ON M5G, Canada]","[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",0,[],NaN
2,e-0-5db70a0c306d6b000861ac9f-2,0,"[{'summary': 'This spot is popular', 'type': '...",5db70a0c306d6b000861ac9f,NEO COFFEE BAR,770 Bay Street Unit 3,NaN,43.660130,-79.385830,"[{'label': 'display', 'lat': 43.66013, 'lng': ...",...,M5G 0A6,CA,Toronto,ON,Canada,"[770 Bay Street Unit 3, Toronto ON M5G 0A6, Ca...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],NaN
3,e-0-52d884c5498ecf5c7cafe5ab-3,0,"[{'summary': 'This spot is popular', 'type': '...",52d884c5498ecf5c7cafe5ab,Nando's,832 Bay St,Grenville St,43.661728,-79.386391,"[{'label': 'display', 'lat': 43.66172826, 'lng...",...,M5S 1Z6,CA,Toronto,ON,Canada,"[832 Bay St (Grenville St), Toronto ON M5S 1Z6...","[{'id': '4def73e84765ae376e57713a', 'name': 'P...",0,[],550386889
4,e-0-594ff53c2be42528bcc1bdb7-4,0,"[{'summary': 'This spot is popular', 'type': '...",594ff53c2be42528bcc1bdb7,T-Swirl Crepe,510 Yonge Street,NaN,43.663452,-79.384125,"[{'label': 'display', 'lat': 43.663452, 'lng':...",...,M4Y 1X9,CA,Toronto,ON,Canada,"[510 Yonge Street, Toronto ON M4Y 1X9, Canada]","[{'id': '52e81612bcbc57f1066b79f2', 'name': 'C...",0,[],455073193
5,e-0-5d9399e49b61d90008bac7b0-5,0,"[{'summary': 'This spot is popular', 'type': '...",5d9399e49b61d90008bac7b0,Bar Volo,17 St. Nicholas St.,NaN,43.665462,-79.385692,"[{'label': 'display', 'lat': 43.66546184849369...",...,M4Y 3G4,CA,Toronto,ON,Canada,"[17 St. Nicholas St., Toronto ON M4Y 3G4, Canada]","[{'id': '56aa371ce4b08b9a8d57356c', 'name': 'B...",0,[],NaN
6,e-0-4aeb5989f964a5203ec121e3-6,0,"[{'summary': 'This spot is popular', 'type': '...",4aeb5989f964a5203ec121e3,The Yoga Sanctuary,"2 College St. W, Suite 306",at Yonge St.,43.661499,-79.383636,"[{'label': 'display', 'lat': 43.66149949759759...",...,M5G 1K3,CA,Toronto,ON,Canada,"[2 College St. W, Suite 306 (at Yonge St.), To...","[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",0,[],NaN
7,e-0-4ad8d773f964a5203e1521e3-7,0,"[{'summary': 'This spot is popular', 'type': '...",4ad8d773f964a5203e1521e3,Central YMCA,20 Grosvenor St,btwn St. Vincent Ln & St. Lukes Ln,43.663083,-79.385025,"[{'label': 'display', 'lat': 43.66308331006765...",...,M4Y 2V5,CA,Toronto,ON,Canada,[20 Grosvenor St (btwn St. Vincent Ln & St. Lu...,"[{'id': '52e81612bcbc57f1066b7a37', 'name': 'D...",0,[],NaN
8,e-0-4d927b8352ed224b27d008ac-8,0,"[{'summary': 'This spot is popular', 'type': '...",4d927b8352ed224b27d008ac,Burrito Bandidos,496 Yonge St.,at Grosvenor St.,43.662962,-79.383956,"[{'label': 'display', 'lat': 43.66296217816934...",...,M4Y 1X9,CA,Toronto,ON,Canada,"[496 Yonge St. (at Grosvenor St.), Toronto ON ...","[{'id': '4bf58dd8d48988d153941735', 'name': 'B...",0,[],NaN
9,e-0-4fb593c0e4b05a76e2ab2951-9,0,"[{'summary': 'This spot is popular', 'type': '...",4fb593c0e4b05a76e2ab2951,Como En Casa,565 Yonge St,NaN,43.665160,-79.384796,"[{'label': 'display', 'lat': 43.66516, 'lng': ...",...,M4Y-1Z2,CA,Toronto,ON,

In [21]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Queen's Park,Park,43.663946,-79.392180
1,Mercatto,Italian Restaurant,43.660391,-79.387664
2,NEO COFFEE BAR,Coffee Shop,43.660130,-79.385830
3,Nando's,Portuguese Restaurant,43.661728,-79.386391
4,T-Swirl Crepe,Creperie,43.663452,-79.384125


In [23]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

30 venues were returned by Foursquare.


In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
venues = getNearbyVenues(names=df_concat['Borough'],latitudes=df_concat['Latitude'],longitudes=df_concat['Longitude'])

North York
North York
Downtown Toronto
North York
Downtown Toronto
Etobicoke
Scarborough
North York
East York
Downtown Toronto
North York
Etobicoke
Scarborough
North York
East York
Downtown Toronto
York
Etobicoke
Scarborough
East Toronto
Downtown Toronto
York
Scarborough
East York
Downtown Toronto
Downtown Toronto
Scarborough
North York
North York
East York
Downtown Toronto
West Toronto
Scarborough
North York
North York
East York
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
North York
North York
Scarborough
North York
North York
East Toronto
North York
York
North York
Scarborough
North York
North York
Central Toronto
Central Toronto
Toronto/York
York
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Etobicoke
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Mississauga
Etobicoke
Scarborough
Central Toronto
Downtown Toronto
We

In [27]:
print('There are {} unique categories.'.format(len(venues['Venue Category'].unique())))

There are 272 unique categories.


In [26]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

onehot.head()

,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008772,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.008772,0.000000,0.000000,0.000000,0.000000,0.008772
1,Downtown Toronto,0.000000,0.000819,0.000819,0.001638,0.002457,0.001638,0.010647,0.001638,0.004095,...,0.002457,0.012285,0.000819,0.000000,0.003276,0.000000,0.006552,0.000000,0.000000,0.005733
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025210,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025210
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.012346,0.000000,0.012346,0.000000,0.000000,0.000000,0.012346
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014286,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014286,0.000000,0.000000
5,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.004115,0.004115,0.000000,0.000000,0.000000,0.000000,0.008230,0.000000,0.000000,...,0.000000,0.000000,0.004115,0.000000,0.008230,0.000000,0.000000,0.000000,0.004115,0.000000
7,Scarborough,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010870,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.021739,0.000000,0.000000,0.000000,0.000000,0.000000
8,Toronto/York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006410,0.000000,...,0.000000,0.019231,0.000000,0.000000,0.006410,0.000000,0.006410,0.000000,0.000000,0.012821


In [31]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1  Sandwich Place  0.06
2            Café  0.05
3     Pizza Place  0.05
4            Park  0.05


----Downtown Toronto----
                 venue  freq
0          Coffee Shop  0.10
1                 Café  0.05
2                Hotel  0.03
3  Japanese Restaurant  0.03
4           Restaurant  0.03


----East Toronto----
                venue  freq
0    Greek Restaurant  0.07
1         Coffee Shop  0.06
2  Italian Restaurant  0.04
3             Brewery  0.04
4         Yoga Studio  0.03


----East York----
                 venue  freq
0          Coffee Shop  0.07
1                 Park  0.05
2          Pizza Place  0.05
3                 Bank  0.05
4  Sporting Goods Shop  0.04


----Etobicoke----
            venue  freq
0     Pizza Place  0.10
1  Sandwich Place  0.07
2     Coffee Shop  0.06
3        Pharmacy  0.04
4          Bakery  0.04


----Mississauga----
            venue  freq
0     Coffee Shop  0.21
1           

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [90]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Borough'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Cluster Labels,Coffee Shop,Sandwich Place,Park,Pizza Place,Café,Clothing Store,Sushi Restaurant,Dessert Shop,Gym
1,Downtown Toronto,Cluster Labels,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Seafood Restaurant,Park
2,East Toronto,Cluster Labels,Greek Restaurant,Coffee Shop,Italian Restaurant,Brewery,Ice Cream Shop,Park,Fast Food Restaurant,American Restaurant,Bakery
3,East York,Cluster Labels,Coffee Shop,Park,Bank,Pizza Place,Sandwich Place,Intersection,Sporting Goods Shop,Burger Joint,Grocery Store
4,Etobicoke,Cluster Labels,Pizza Place,Sandwich Place,Coffee Shop,Café,Grocery Store,Fast Food Restaurant,Bakery,Pharmacy,Gym
5,Mississauga,Cluster Labels,Coffee Shop,Hotel,Gym,Middle Eastern Restaurant,Fried Chicken Joint,Gas Station,Sandwich Place,Intersection,American Restaurant
6,North York,Cluster Labels,Coffee Shop,Clothing Store,Pizza Place,Restaurant,Bank,Sandwich Place,Fast Food Restaurant,Park,Japanese Restaurant
7,Scarborough,Fast Food Restaurant,Coffee Shop,Bank,Bakery,Intersection,Breakfast Spot,Discount Store,Chinese Restaurant,Pharmacy,Pizza Place
8,Toronto/York,Cluster Labels,Breakfast Spot,Grocery Store,Caribbean Restaurant,Bus Line,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
9,West Toronto,Cluster Labels,Bar,Café,Coffee Shop,Restaurant,Italian Restaurant,Bakery,Grocery Store,Breakfast Spot,Park


In [36]:
# set number of clusters
kclusters = 7

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([5, 4, 4, 5, 6, 3, 5, 0, 2, 4])

In [86]:
data = df_concat[['Borough','Latitude','Longitude']]

data=data.groupby('Borough').agg('mean')


In [97]:
merged = data

# add clustering labels
merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Borough'), on='Borough',how='right')

merged.reset_index(inplace=True)
merged.head()# check the last columns!

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,43.701980,-79.398954,5,Cluster Labels,Coffee Shop,Sandwich Place,Park,Pizza Place,Café,Clothing Store,Sushi Restaurant,Dessert Shop,Gym
1,Downtown Toronto,43.654597,-79.383972,4,Cluster Labels,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Seafood Restaurant,Park
2,East Toronto,43.669436,-79.324654,4,Cluster Labels,Greek Restaurant,Coffee Shop,Italian Restaurant,Brewery,Ice Cream Shop,Park,Fast Food Restaurant,American Restaurant,Bakery
3,East York,43.700303,-79.335851,5,Cluster Labels,Coffee Shop,Park,Bank,Pizza Place,Sandwich Place,Intersection,Sporting Goods Shop,Burger Joint,Grocery Store
4,Etobicoke,43.660043,-79.542074,6,Cluster Labels,Pizza Place,Sandwich Place,Coffee Shop,Café,Grocery Store,Fast Food Restaurant,Bakery,Pharmacy,Gym


In [100]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [101]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Borough'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters